In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import pandas_profiling as pp

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
st = pd.read_csv(r"../input/predict-test-scores-of-students/test_scores.csv")

In [ ]:
st.head()

In [ ]:
pp.ProfileReport(st)

In [ ]:
st.isna().sum()

***There is no missing values***

## Task1:let us check whether scool_setting matter for the students  score or not??

In [ ]:
plt.figure(figsize= (30,20))
plt.subplot(331)
plt.scatter(st.school_setting,st.pretest)
plt.ylabel('Marks')
plt.xlabel('school_setting')
plt.title('school_setting vs pretest')
plt.subplot(332)
plt.scatter(st.school_setting,st.posttest)
plt.ylabel('Marks')
plt.xlabel('school_setting')
plt.title('school_setting vs prosttest ')

In [ ]:
st.groupby('school_setting').agg({'pretest':[max,min,'mean','mad'],'posttest':[max,min,'mean','mad']}).round(3)

**The performance of students from Suburban is good in both pretest and posttest while compared to the students from  Rural and Urban.**

## Task2:let us check whether school_type matters for score or not?

In [ ]:
plt.figure(figsize= (30,20))
plt.subplot(331)
plt.scatter(st.school_type,st.pretest)
plt.ylabel('Marks')
plt.xlabel('school_type')
plt.title('school_type vs pretest')
plt.subplot(332)
plt.scatter(st.school_type,st.posttest)
plt.ylabel('Marks')
plt.xlabel('school_type')
plt.title('school_type vs prosttest ')

In [ ]:

st.groupby('school_type').agg({'pretest':[max,min,'mean','mad'],'posttest':[max,min,'mean','mad']}).round(3)

<b>students studying in non-public schools have good score then compared to the student studying in public schools</b>

## Task3:which teaching_method is more effective  for students?? 

In [ ]:
st.groupby('teaching_method').agg({'pretest':[max,min,'mean','mad'],"posttest":[max,min,'mean','mad']})

In [ ]:
#lets plot this result using plot.
sns.lmplot(data=st, x="posttest", y="pretest", hue="teaching_method",height = 10, aspect = 1)


<b>from above result we can tell  standard teaching is better than experimental.</b>
- <i>based on mad </i> 

## Task4: let us find out avg scores of the students who are qualified for the free lunch?

    - Here we are trying to find,do we have any such relation b/w free lunch and there score.

In [ ]:
st.lunch.unique()

In [ ]:
st.value_counts(['lunch','gender']) 

In [ ]:

st.groupby('lunch').agg({'pretest':[max,min,'mean','mad'],"posttest":[max,min,'mean','mad'] ,"lunch":'count'})

In [ ]:
#creating a column of sum of pretest and posttest to find there avg.
st['total_score'] = st['pretest'] + st['posttest']

In [ ]:
st.head(5)

In [ ]:
## creating the column of avg scores of the students in pretest and posttest.
st['avg_score'] = st['total_score']/2

In [ ]:
st.head()

In [ ]:
def st_pass(x):
    if x >= 85:
        return 'A+'
    elif x >= 75:
        return 'A'
    elif x >= 65:
        return 'B'
    elif x >= 50:
        return 'C'
    elif x >= 35:
        return 'D'
    else:
        return 'F'

In [ ]:
st['Rank'] = st['avg_score'].apply(st_pass)

In [ ]:
st.head()

In [ ]:
def pass_fail(x):
    if x >= 35:
        return 'pass'
    else:
        return 'fail'

In [ ]:
st['pass/fail'] = st['avg_score'].apply(pass_fail)

In [ ]:
st.head()

In [ ]:
st.groupby(['Rank','lunch','pass/fail']).agg({'lunch':'count'})

**From the above result we can tell that,The students who belongs to the category Rank of D & C are qualified more for free lunch or reduced lunch so we are clear that there is no such relationship b/w students score and the students qualified for reduced lunch**

## Task5: Who  scored more in  both the  test??

In [ ]:
st.groupby(['gender']).agg({'total_score': 'mean'})

In [ ]:
sns.barplot(x = 'gender', y = 'total_score' , data = st , hue = 'gender')

*****The avg_scores of  male & female doesn't vary much which means they are most same*****

# Task6:what are the top 5 schools in ranking ??

In [ ]:
st.groupby(['school','school_type']).agg({'avg_score':max,'n_student': 'count'}).sort_values(by = 'avg_score',ascending = False).head(5)

# Task7:Find the student_id of the students who scored top in thr specific rank category ??

In [ ]:
st.groupby(['Rank']).agg({'student_id':max,'avg_score':max}).sort_values(by = 'avg_score', ascending = False)

## overall graph

In [ ]:
#default theme
sns.set_theme()

# Create a visualization
sns.relplot(
    data=st,
    x="posttest", y="pretest",size = 'school_setting',style = 'school_type'
    ,hue="teaching_method")